In [1]:
import boto3
import argparse
import time
import os
import trp
import json
import re
import csv

In [2]:
# In the python terminal
#ap = argparse.ArgumentParser()
#ap.add_argument("-a", "--file", required=True,
#   help="json file da analizare")
#nomefile = vars(ap.parse_args())
#args = vars(ap.parse_args())
#with open('{}'.format(nomefile['file']), 'r') as openfile:
  
    # Reading from json file
#    json_object = json.load(openfile)
  
#response = list(json_object)

In [3]:
# In jupiter notebook
with open('genialloyd.json', 'r') as openfile:
  
# Reading from json file
    json_object = json.load(openfile)
  
response = list(json_object)


In [4]:
import csv
with open('policytractormask.csv') as csvfile:
    mask = list(csv.DictReader(csvfile))

In [5]:
openfile.close()
csvfile.close()

# Estimate of inclination angle of pdf file

In [6]:
# estimates the tangent of the angle starting from the y coordinate of two following words

def inclination(rispostajson):
    
    for resultPage in response:
        for i, item in enumerate(resultPage["Blocks"]):
            if item['BlockType']== 'LINE':
                # look for a line with more than one word
               findline = len (item['Text'].split())
           
               if findline >3:
                    a = (item['Relationships'][0]['Ids'])
                                          
                
                # exit cycle if line is  found
                    break

    inclinationcoord = {'left0': '', 'top0': '', 'left1': '', 'top1': ''}
    
    for resultPage in response:
        for i, item in enumerate(resultPage["Blocks"]):
            if item['BlockType']== 'WORD':
                if item['Id']== a[0]:
                    inclinationcoord['left0'], inclinationcoord['top0'] = item['Geometry']['BoundingBox']['Left']+item['Geometry']['BoundingBox']['Width']/2, item['Geometry']['BoundingBox']['Top']
                if item['Id']== a[len(a)-1]:
                    inclinationcoord['left1'], inclinationcoord['top1'] = item['Geometry']['BoundingBox']['Left']+item['Geometry']['BoundingBox']['Width']/2, item['Geometry']['BoundingBox']['Top']
    
    
    taninclination = (inclinationcoord['top1']-inclinationcoord['top0'])/(inclinationcoord['left1']-inclinationcoord['left0'])        
    
    return taninclination
    

In [7]:
taninclination = inclination(response)

In [8]:
def ricalcolocoordy (dictcoordinates, taninclination):
    for key, coordinates in dictcoordinates.items():
        dictcoordinates[key] = (coordinates[0], coordinates[1]-coordinates[0]*taninclination)
    return dictcoordinates

# Generate full text of OCR from Json

In [9]:
text = []
for page, pageResult in enumerate(response):
    for i, item in enumerate(pageResult["Blocks"]):
        if item["BlockType"] == "WORD":
            
            text.append(item['Text'].lower())

In [10]:
companies = set(d['compagnia'] for d in mask)

## Identify the company of the template

In [11]:
d = dict()

# calculate the frequency of the name of a certain insurance company in the text extracted by OCR
for company in companies:

    list_company = [w for w in text if w == company]
    n_company = len(list_company)
    d[company] = n_company

In [12]:
# takes the most frequent company name as the name of the comapny to be used in the csv input mask
dsorted = (sorted(d.items(), reverse=True, key = lambda kv:(kv[1], kv[0])))
compagnia = dsorted[0][0]

# Find the date of starting the policy and the expiration of the policy
## In the future for additional templates, to be included the  regular expression for other date formats


In [14]:
# a dictionary with dates and their coordinates is generated by the function 
def coord_date (jObjectResponse):


    coord_date = {}

    for resultPage in jObjectResponse:
        for i, item in enumerate(resultPage["Blocks"]):
            if item["BlockType"] == "WORD":
            
                m = re.search(r"^(?P<day>[0-3]?[0-9])[/-](?P<month>[0-3]?[0-9])[/-](?:[0-9]{2})?(?P<year>[0-9]{2})[,.]?$", item["Text"])

                if m is not None:
                    if item["BlockType"] == "WORD":
                   
                        coord_date [item["Page"], i, item['Text']] = (item['Geometry']['BoundingBox']['Left']+item['Geometry']['BoundingBox']['Width']/2, item['Geometry']['BoundingBox']['Top'])

# generato un dizionario in cui si riportano le date sul documento e le rispettive coordinate                    
    return coord_date

In [15]:
# per ogni data le parole chiave
word_date =[]
word_date = [d['data'] for d in mask if d['compagnia'] == compagnia and d['data']!='']

In [16]:
# generiamo un dizionaio con le possibili descrizioni delle date e le loro coordinate 
def coord_keyworddate (keyword_date, jObjectResponse):

    coord_keyworddate = {}

    for resultPage in jObjectResponse:
        for i, item in enumerate(resultPage["Blocks"]):
            if item["BlockType"] == "LINE":
                for keyword in keyword_date:
                    m = re.search(keyword, item["Text"])
                    if m is not None:   
                        coord_keyworddate [(item["Page"], i, keyword)] =(item['Geometry']['BoundingBox']['Left']+item['Geometry']['BoundingBox']['Width']/2, item['Geometry']['BoundingBox']['Top'])

# generato un dizionario in cui si riportano le date sul documento e le rispettive coordinate                    

    return coord_keyworddate

In [17]:
coord_date = coord_date (response)
coord_keyworddate = coord_keyworddate (word_date, response)

In [18]:
new_coord_keyworddate = ricalcolocoordy (coord_keyworddate, taninclination)
new_coord_date = ricalcolocoordy (coord_date, taninclination)

In [19]:
for descrdate, coorddescrdate in new_coord_keyworddate.items():
    for date, coorddate in new_coord_date.items():
        if abs((coorddescrdate[1] - coorddate[1]))<0.009:
            if descrdate[0] == date[0]:
                sottrazione = (coorddescrdate[1] - coorddate[1])
                print(descrdate[2], ":", date[2]) #, ":", coorddate[1], sottrazione)
            # una volta trovato un match di tipo valore (data in questo caso) e parola chiave, esce dal loop
                break
        # se non trova un valore alluineato, allora prova a verificare entro le due linee sottostanti
        elif abs((coorddescrdate[0] - coorddate[0]))<0.05 and abs((coorddescrdate[1] - coorddate[1]))<0.030:
            if descrdate[0] == date[0]:
                sottrazione = (coorddescrdate[1] - coorddate[1])
                print(descrdate[2], ":", date[2]) #, ":", coorddate[1], sottrazione)

Valido fino al : 24/05/2020
Decorrenza richiesta : 01/04/2020
Scadenza : 01/04/2021


# Lista delle garanzie in base a compagnia

In [20]:
garanzie = [d['garanzie'] for d in mask if d['compagnia'] == compagnia ]
gminx = [d['minx'] for d in mask if d['compagnia'] == compagnia ]
gmaxx = [d['maxx'] for d in mask if d['compagnia'] == compagnia ]
coordlimitgaranzie = [d['deltaYgaranzie'] for d in mask if d['compagnia'] == compagnia and d['deltaYgaranzie']!='']


# Dizionario delle aree della pagina dove si trovano i premi delle  garanzie

In [21]:
dictgaranzie = {garanzie[i]: (gminx[i], gmaxx[i]) for i in range(len(garanzie))}
dictgaranzie

{"Premio per l'annualità": ('0.85', '0.95'),
 'RC \\+ Assistenza Stradale': ('0.85', '0.95'),
 'Bonus Protetto': ('0.85', '0.95'),
 '^Furto e Incendio': ('0.85', '0.95'),
 'Estensioni': ('0.85', '0.95'),
 'Eventi Naturali': ('0.85', '0.95'),
 'Collisione con veicolo non identificato': ('0.85', '0.95'),
 'Cristalli': ('0.85', '0.95'),
 'Infortunio Guidatore': ('0.85', '0.95'),
 'Assistenza Stradale Plus': ('0.85', '0.95'),
 'Assistenza Stradale ZeroPensieri': ('0.85', '0.95'),
 'Tutela Legale': ('0.85', '0.95'),
 'Premi lordi': ('0.85', '0.95')}

In [22]:
# The jason repoonse generated by AWS is a list of json based on queue generated by the service>
type(response)

list

# Generiamo le coordinate delle garanzie in base al dizionario/ lista di garanzie

In [23]:
def coordinategaranzie (garanzie, jObjectResponse):
    
    
    guaranteecoord = {}
    for garanzia in garanzie:
            for resultPage in jObjectResponse:
                for i, item in enumerate(resultPage["Blocks"]):
                    if item["BlockType"] == "LINE":
                    
                        m = re.search(garanzia.lower(), item["Text"].lower())
                    
                        if m is not None:  
                                
                    
                                    guaranteecoord [item['Page'], garanzia] = (item['Geometry']['BoundingBox']['Left']+item['Geometry']['BoundingBox']['Width']/2, item['Geometry']['BoundingBox']['Top'])



# generato un dizionario in cui si riportano le coordinate della garanzia esplorata                    
    return guaranteecoord

# Premi delle garanzie

In [24]:
# generiamo un dizionaio con i premi e le loro coordinate 
def coordinatepremi (dizionariogaranzie, jObjectResponse):


    premiumcoord = {}

    for garanzia in dizionariogaranzie.keys(): 
        for resultPage in jObjectResponse:
            for i, item in enumerate(resultPage["Blocks"]):
                if item["BlockType"] == "WORD":
            
                    m = re.search(r'^[+-]?(\d\.)?[0-9]{1,3}[.,]+[0-9]{1,2}€?$', item["Text"])

                    if m is not None:
                        if item["BlockType"] == "WORD":
                            if float(dizionariogaranzie[garanzia][0])<item["Geometry"]['BoundingBox']['Left']<float(dizionariogaranzie[garanzia][1]):
                   
                                premiumcoord [item["Page"], item['Text']] = (item['Geometry']['BoundingBox']['Left']+item['Geometry']['BoundingBox']['Width']/2, item['Geometry']['BoundingBox']['Top'])

# generato un dizionario in cui si riportano le coordinate dei premi                    
    return premiumcoord

# Normalizziamo le coordinate degli elementi trovati

In [25]:
guaranteecoord = coordinategaranzie(dictgaranzie, response)
premiumcoord = coordinatepremi(dictgaranzie, response)

In [26]:
guaranteecoord

{(1, "Premio per l'annualità"): (0.7384517788887024, 0.8903414011001587),
 (1, 'RC \\+ Assistenza Stradale'): (0.11429890617728233, 0.6828791499137878),
 (1, 'Bonus Protetto'): (0.1690422110259533, 0.7081286311149597),
 (1, '^Furto e Incendio'): (0.08743909746408463, 0.723648726940155),
 (1, 'Estensioni'): (0.14855100959539413, 0.7389119863510132),
 (1, 'Eventi Naturali'): (0.08146390318870544, 0.7544188499450684),
 (1, 'Collisione con veicolo non identificato'): (0.14942666888237,
  0.7698591947555542),
 (1, 'Cristalli'): (0.06097473204135895, 0.7852249145507812),
 (1, 'Infortunio Guidatore'): (0.09634368494153023, 0.8006728887557983),
 (1, 'Assistenza Stradale Plus'): (0.10879578441381454, 0.8166330456733704),
 (1, 'Assistenza Stradale ZeroPensieri'): (0.1321399137377739,
  0.831530749797821),
 (1, 'Tutela Legale'): (0.07534557580947876, 0.846922755241394),
 (1, 'Premi lordi'): (0.8935849033296108, 0.6553114652633667)}

In [27]:
premiumcoord

{(1, '234,59'): (0.9204914066940546, 0.6866454482078552),
 (1, '0,00'): (0.9228635998442769, 0.7080795168876648),
 (1, '10,00'): (0.9205827005207539, 0.7226750254631042),
 (1, '141,03'): (0.9164853189140558, 0.7381691336631775),
 (1, '12,01'): (0.919026373885572, 0.7538179159164429),
 (1, '41,68'): (0.9183475933969021, 0.7692275643348694),
 (1, '31,60'): (0.9184201192110777, 0.7844710946083069),
 (1, '35,58'): (0.9176366990432143, 0.8000286817550659),
 (1, '24,51'): (0.9168690163642168, 0.8153876662254333),
 (1, '11,00'): (0.9170502321794629, 0.8302255868911743),
 (1, '8,00'): (0.9195563495159149, 0.8457632064819336),
 (1, '13,00'): (0.9161609038710594, 0.8613213896751404),
 (1, '563,00'): (0.913575004786253, 0.8951182961463928)}

In [28]:
new_guaranteecoord = ricalcolocoordy (guaranteecoord, taninclination)
new_premiumcoord = ricalcolocoordy (premiumcoord, taninclination)

In [29]:
new_guaranteecoord

{(1, "Premio per l'annualità"): (0.7384517788887024, 0.8762613488797191),
 (1, 'RC \\+ Assistenza Stradale'): (0.11429890617728233, 0.6806998138843762),
 (1, 'Bonus Protetto'): (0.1690422110259533, 0.7049055050737633),
 (1, '^Furto e Incendio'): (0.08743909746408463, 0.7219815265980349),
 (1, 'Estensioni'): (0.14855100959539413, 0.7360795658297303),
 (1, 'Eventi Naturali'): (0.08146390318870544, 0.7528655785686977),
 (1, 'Collisione con veicolo non identificato'): (0.14942666888237,
  0.7670100780479523),
 (1, 'Cristalli'): (0.06097473204135895, 0.7840623099829357),
 (1, 'Infortunio Guidatore'): (0.09634368494153023, 0.7988359047388761),
 (1, 'Assistenza Stradale Plus'): (0.10879578441381454, 0.8145586376081403),
 (1, 'Assistenza Stradale ZeroPensieri'): (0.1321399137377739,
  0.8290112394668524),
 (1, 'Tutela Legale'): (0.07534557580947876, 0.8454861419481428),
 (1, 'Premi lordi'): (0.8935849033296108, 0.6382734914078338)}

In [30]:
new_premiumcoord

{(1, '234,59'): (0.9204914066940546, 0.6690944483386116),
 (1, '0,00'): (0.9228635998442769, 0.6904832864368357),
 (1, '10,00'): (0.9205827005207539, 0.7051222848954373),
 (1, '141,03'): (0.9164853189140558, 0.720694517827062),
 (1, '12,01'): (0.919026373885572, 0.7362948498121896),
 (1, '41,68'): (0.9183475933969021, 0.7517174405309801),
 (1, '31,60'): (0.9184201192110777, 0.7669595879554895),
 (1, '35,58'): (0.9176366990432143, 0.7825321125662635),
 (1, '24,51'): (0.9168690163642168, 0.7979057344341132),
 (1, '11,00'): (0.9170502321794629, 0.8127401998598307),
 (1, '8,00'): (0.9195563495159149, 0.8282300353379687),
 (1, '13,00'): (0.9161609038710594, 0.8438529594572021),
 (1, '563,00'): (0.913575004786253, 0.8776991712388238)}

# Stampiamo le garanzie e i relativi premi

In [46]:
for premium, coordpremium in new_premiumcoord.items():
    for guarantee, coordguarantee in new_guaranteecoord.items():
        sottrazione = (coordguarantee[1] - coordpremium[1])
        # controlla che sa stesso numero di pagina
        if abs(coordguarantee[1] - coordpremium[1])<0.009 and guarantee[0] == premium[0]:
            print(guarantee[1], ":", premium[1]) #, ":", coordpremium[1], sottrazione)
        #    print('prima cnd vera')
                        
            
        # se non trova un valore allineato, allora prova a verificare entro le due linee sottostanti
        elif abs((coordguarantee[0] - coordpremium[0]))<0.05 and 0<(coordpremium[1]-coordguarantee[1])<float(coordlimitgaranzie[1]):
                if guarantee[0] == premium[0]:
                    sottrazione = (coordguarantee[1] - coordpremium[1])
                    print(guarantee[1], ":", premium[1]) #, ":", coordpremium[1], sottrazione)
#                    print('seconda cnd vera')

Bonus Protetto : 10,00
^Furto e Incendio : 141,03
Estensioni : 12,01
Eventi Naturali : 41,68
Collisione con veicolo non identificato : 31,60
Cristalli : 35,58
Infortunio Guidatore : 24,51
Assistenza Stradale Plus : 11,00
Assistenza Stradale ZeroPensieri : 8,00
Tutela Legale : 13,00
Premio per l'annualità : 563,00
